In [10]:
from datasets import load_dataset, load_metric

# Dataset is downloaded from https://www.openslr.org/12
data = load_dataset("LibriSpeech")

Resolving data files:   0%|          | 0/585 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/87 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/97 [00:00<?, ?it/s]

Using custom data configuration LibriSpeech-49d9aec4ade7f44e


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /home/kmb85/.cache/huggingface/datasets/text/LibriSpeech-49d9aec4ade7f44e/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
# from parse_data import parse_to_json
import json
import os

# parse_to_json("./LibriSpeech/dev-clean")
# os.rename('data.json', 'dev-clean.json')
with open('dev-clean.json', 'r') as file:
    dev_clean_data = json.load(file)

# parse_to_json("./LibriSpeech/test-clean")
# os.rename('data.json', 'test-clean.json')
with open('test-clean.json', 'r') as file:
    test_clean_data = json.load(file)

In [12]:
dev_clean_dict = {}
for key, value in dev_clean_data.items():
    new_key = key.replace('-', '')
    dev_clean_dict[new_key] = value

In [13]:
test_clean_dict = {}
for key, value in test_clean_data.items():
    new_key = key.replace('-', '')
    test_clean_dict[new_key] = value

In [14]:
import soundfile as sf
import resampy

def flac_to_array_with_sampling_rate(file_path, target_sr=16000):
    data, sr = sf.read(file_path, dtype='float32')

    if sr != target_sr:
        data = resampy.resample(data, sr, target_sr)

    return data

In [15]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters_dev(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    trimmed_string = batch["text"].strip()
    words = trimmed_string.split()
    new_text = ' '.join(words[1:])
    batch["text"] = new_text
    batch["input_values"] = flac_to_array_with_sampling_rate(dev_clean_dict[words[0]]['file_path'])
    return batch

In [16]:
data['validation'] = data['validation'].map(remove_special_characters_dev)

0ex [00:00, ?ex/s]

In [17]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters_test(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    trimmed_string = batch["text"].strip()
    words = trimmed_string.split()
    new_text = ' '.join(words[1:])
    batch["text"] = new_text
    batch["input_values"] = flac_to_array_with_sampling_rate(test_clean_dict[words[0]]['file_path'])
    return batch

In [18]:
data['test'] = data['test'].map(remove_special_characters_test)

0ex [00:00, ?ex/s]

In [19]:
from speechbrain.pretrained import EncoderASR
import torch

In [ ]:
asr_model = EncoderASR.from_hparams(source="speechbrain/asr-wav2vec2-librispeech", savedir="pretrained_models/asr-wav2vec2-librispeech", run_opts={"device":"cuda"})

In [20]:
import pytorch_quantization
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib

In [21]:
model_quantized = EncoderASR.from_hparams(source="speechbrain/asr-wav2vec2-librispeech", savedir="pretrained_models/asr-wav2vec2-librispeech", run_opts={"device":"cuda"})

hyperparams.yaml:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

/home/kmb85/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model ch

wav2vec2.ckpt:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

asr.ckpt:   0%|          | 0.00/8.53M [00:00<?, ?B/s]

tokenizer.ckpt:   0%|          | 0.00/426 [00:00<?, ?B/s]

In [22]:
quant_desc_input = QuantDescriptor(calib_method='max', num_bits=8, fake_quant=True)
quant_nn.QuantConv1d.set_default_quant_desc_input(quant_desc_input)
quant_nn.QuantLinear.set_default_quant_desc_input(quant_desc_input)

In [23]:
quant_modules.initialize()
model_full_quantized = EncoderASR.from_hparams(source="speechbrain/asr-wav2vec2-librispeech", savedir="pretrained_models/asr-wav2vec2-librispeech", run_opts={"device":"cuda"})

Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
W0111 02:43:52.038621 140590214230016 huggingface_wav2vec.py:146] speechbrain.lobes.models

In [24]:
model_full_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].conv._weight_quantizer.enable_calib()
model_full_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].conv._weight_quantizer.disable_quant()

In [25]:
import torch
with torch.no_grad():
    for i in range(2000):
            _ = model_full_quantized.mods.encoder.wav2vec2.model(torch.tensor(data["validation"][i:i+1]["input_values"], device="cuda"))

In [26]:
# Finalize calibration

model_full_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].conv._weight_quantizer.load_calib_amax()
model_full_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].conv._weight_quantizer.disable_calib()
model_full_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].conv._weight_quantizer.enable_quant()

model_full_quantized.cuda()

W0111 02:47:31.099232 140590214230016 tensor_quantizer.py:238] Load calibrated amax, shape=torch.Size([512, 1, 1]).
W0111 02:47:31.100970 140590214230016 tensor_quantizer.py:239] Call .cuda() if running on GPU after loading calibrated amax.
W0111 02:47:31.101726 140590214230016 tensor_quantizer.py:174] Disable MaxCalibrator


EncoderASR(
  (mods): ModuleDict(
    (encoder): LengthsCapableSequential(
      (wav2vec2): HuggingFaceWav2Vec2(
        (model): Wav2Vec2Model(
          (feature_extractor): Wav2Vec2FeatureEncoder(
            (conv_layers): ModuleList(
              (0): Wav2Vec2LayerNormConvLayer(
                (conv): QuantConv1d(
                  1, 512, kernel_size=(10,), stride=(5,)
                  (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
                  (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=[0.0071, 0.6079](512) calibrator=MaxCalibrator scale=1.0 quant)
                )
                (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (activation): GELUActivation()
              )
              (1-4): 4 x Wav2Vec2LayerNormConvLayer(
                (conv): QuantConv1d(
                  512, 512, kernel_size=(3,), stride=(2,)
                  (_input_quantizer

In our final quantized model, we only keep the quantized first conv layer.

In [27]:
model_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].conv = model_full_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].conv

In [37]:
from typing import Tuple


class AdaroundHyperParameters:
    """
    Hyper parameters for Adaround
    """
    def __init__(self, num_iterations: int, reg_param: float, beta_range: Tuple, warm_start: float):
        """
        :param num_iterations: Number of maximum iterations to adaround layer
        :param reg_param: Regularization parameter, trading off between rounding loss vs reconstruction loss
        :param beta_range: Start and stop parameters for annealing of rounding loss (start_beta, end_beta)
        :param warm_start: Warm up period, during which rounding loss has zero effect
        """
        self.num_iterations = num_iterations
        self.reg_param = reg_param
        self.beta_range = beta_range
        self.warm_start = warm_start

class AdaroundLoss:
    """
    Calculates the Reconstruction loss and Rounding loss - needed for Adaround optimization to
    learn weight rounding
    """
    @staticmethod
    def compute_recon_loss(ada_quantized_output: torch.Tensor, orig_output: torch.Tensor) -> torch.Tensor:
        """
        Compute Reconstruction Loss using Squared Frobenius Norm - first part of Combined Loss
        :param ada_quantized_output: Activation output from quantized wrapper module
        :param orig_output: Activation output from original module
        :return: reconstruction loss
        """
        recon_loss = (torch.norm(ada_quantized_output - orig_output, p="fro", dim=1) ** 2).mean()

        return recon_loss

    @classmethod
    def compute_round_loss(cls, alpha: torch.Tensor, opt_params: AdaroundHyperParameters,
                           cur_iter: int) -> torch.Tensor:
        """
        Compute Rounding Loss - second part of Combined Loss
        :param alpha: parameter 'alpha' to be optimized, float32 tensor same shape as weight tensor
        :param opt_params: Optimization parameters for Adaround
        :param cur_iter: current iteration
        :return: rounding loss
        """
        if cur_iter < opt_params.num_iterations * opt_params.warm_start:
            # Warm Start duration
            round_loss = 0

        else:
            # compute rectified sigmoid of parameter 'alpha' which maps it between zero and one
            h_alpha = torch.clamp(torch.sigmoid(alpha) * (AdaroundConstants.ZETA - AdaroundConstants.GAMMA) +
                                  AdaroundConstants.GAMMA, 0, 1)

            # compute beta parameter to anneal the rounding loss
            beta = cls._compute_beta(opt_params.num_iterations, cur_iter, opt_params.beta_range, opt_params.warm_start)

            # calculate regularization term - which ensures parameter to converge to exactly zeros and ones
            # at the end of optimization
            reg_term = torch.add(1, -(torch.add(2 * h_alpha, -1).abs()).pow(beta)).sum()

            # calculate the rounding loss
            round_loss = opt_params.reg_param * reg_term

        return round_loss

    @staticmethod
    def _compute_beta(max_iter: int, cur_iter: int, beta_range: Tuple, warm_start: float) -> float:
        """
        Compute beta parameter used in regularization function using cosine decay
        :param max_iter: total maximum number of iterations
        :param cur_iter: current iteration
        :param beta_range: range for beta decay (start_beta, end_beta)
        :param warm_start: warm up period, during which rounding loss has zero effect
        :return: parameter beta
        """
        assert cur_iter < max_iter, 'Current iteration should be less than total maximum number of iterations.'

        #  Start and stop beta for annealing of rounding loss (start_beta, end_beta)
        start_beta, end_beta = beta_range

        # iteration at end of warm start period, which is 20% of max iterations
        warm_start_end_iter = warm_start * max_iter

        # compute relative iteration of current iteration
        rel_iter = (cur_iter - warm_start_end_iter) / (max_iter - warm_start_end_iter)
        beta = end_beta + 0.5 * (start_beta - end_beta) * (1 + np.cos(rel_iter * np.pi))

        return beta

class AdaroundConstants:
    """ Constants used for Adarounding """

    GAMMA = -0.1
    ZETA = 1.1


def optimize_adaround(layer, norm, act, dataset, device):
    tensor = layer.weight.data
    amax = layer.weight_quantizer.amax
    unsigned = layer.weight_quantizer.unsigned
    num_bits = layer.weight_quantizer.num_bits
    max_bound = torch.tensor((2.0 ** (num_bits - 1 + int(unsigned))) - 1.0, device=amax.device)

    scale = max_bound / amax

    delta = layer.weight_quantizer.amax
    tensor_floor = torch.floor(tensor * scale)
    tensor = (tensor*scale) - tensor_floor

    alpha = - torch.log((AdaroundConstants.ZETA - AdaroundConstants.GAMMA) / (tensor - AdaroundConstants.GAMMA) - 1)
    alpha = torch.nn.Parameter(alpha.float(), requires_grad=True)
    alpha = alpha.to(device=device, dtype=tensor.dtype)

    num_iterations = 4000  # Example value, can be tuned
    reg_param = 0.01  # Example value, can be tuned
    beta_range = (20, 2)  # Example values, can be tuned
    warm_start = 0.05  # Example value, can be tuned
    opt_params = AdaroundHyperParameters(num_iterations, reg_param, beta_range, warm_start)

    optimizer = torch.optim.Adam([alpha], lr=0.01)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=1)

    for iteration in tqdm(range(num_iterations), desc="Adaround Optimization"):
        total_loss = 0.0

        for inputs in data_loader:
            inputs = inputs.to(device)
            optimizer.zero_grad()

            orig_output = forward_pass(layer, norm, act, inputs)
            quant_output = adaround_forward_pass(layer, norm, act, inputs, alpha, device)

            # Calculate losses
            loss = compute_loss(orig_output, quant_output, alpha, opt_params, iteration)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Print the average loss for the current iteration
        avg_loss = total_loss / len(data_loader)
        tqdm.write(f"Iteration {iteration+1}/{num_iterations}, Average Loss: {avg_loss}")

    return alpha.detach()

In [38]:
import logging

def forward_pass(quant_layer, norm, act, inputs):
    weights = quant_layer.weight.data
    bias = quant_layer.bias.data if quant_layer.bias is not None else None

    if isinstance(quant_layer, quant_nn.QuantConv1d):
        conv_output = torch.nn.functional.conv1d(inputs, weights, bias, quant_layer.stride,
                                                 quant_layer.padding, quant_layer.dilation, quant_layer.groups)
        conv_output = conv_output.transpose(-2, -1)

        if norm is not None:
            for param in norm.parameters():
                param.requires_grad = False
            norm_output = norm(conv_output)
            norm_output = norm_output.transpose(-2, -1)
        else:
            norm_output = conv_output

    if act is not None:
        act_output = act(norm_output)
    else:
        act_output = norm_output

    return act_output



def adaround_forward_pass(quant_layer, norm, act, inputs, alpha, device):

    original_weights = quant_layer.weight.data.detach()
    bias = quant_layer.bias.data.detach() if quant_layer.bias is not None else None


    adaround_weights = adaround_weights_fn(original_weights, alpha, device, quant_layer.weight_quantizer.amax,
                                           narrow_range=quant_layer.weight_quantizer.narrow_range)

    #quant_inputs = quant_layer.input_quantizer(inputs)


    if isinstance(quant_layer, quant_nn.QuantConv1d):
        #conv_output = torch.nn.functional.conv2d(quant_inputs, adaround_weights, bias, quant_layer.stride,
                                                 #quant_layer.padding, quant_layer.dilation, quant_layer.groups)
        conv_output = torch.nn.functional.conv1d(inputs, adaround_weights, bias, quant_layer.stride,
                                                 quant_layer.padding, quant_layer.dilation, quant_layer.groups)
        conv_output = conv_output.transpose(-2, -1)

        if norm is not None:
            for param in norm.parameters():
                param.requires_grad = False
            norm_output = norm(conv_output)
            norm_output = norm_output.transpose(-2, -1)
        else:
            norm_output = conv_output

    if act is not None:
        act_output = act(norm_output)
    else:
        act_output = norm_output

    return act_output



def adaround_weights_fn(inputs, alpha, device, amax, num_bits=8, unsigned=False, narrow_range=True,):
    if isinstance(amax, torch.Tensor) and inputs.dim() != amax.dim():
        logging.debug("amax %s has different shape than inputs %s. Make sure broadcast works as expected!",
                      amax.size(), inputs.size())

    logging.debug("{} bits quantization on shape {} tensor.".format(num_bits, inputs.size()))

    if unsigned:
        if inputs.min() < 0.:
            raise TypeError("Negative values encountered in unsigned quantization.")

    # Computation must be in FP32 to prevent potential over flow.
    input_dtype = inputs.dtype
    if inputs.dtype == torch.half:
        inputs = inputs.float()
    if amax.dtype == torch.half:
        amax = amax.float()

    min_amax = amax.min()
    if min_amax < 0:
        raise ValueError("Negative values in amax")

    max_bound = torch.tensor((2.0 ** (num_bits - 1 + int(unsigned))) - 1.0, device=amax.device)
    if unsigned:
        min_bound = 0
    elif narrow_range:
        min_bound = -max_bound
    else:
        min_bound = -max_bound - 1
    scale = max_bound / amax

    epsilon = 1. / (1 << 24)
    if min_amax <= epsilon:  # Treat amax smaller than minimum representable of fp16 0
        zero_amax_mask = (amax <= epsilon)
        scale[zero_amax_mask] = 0  # Value quantized with amax=0 should all be 0

    h_alpha = torch.sigmoid(alpha) * (AdaroundConstants.ZETA - AdaroundConstants.GAMMA) + AdaroundConstants.GAMMA
    h_alpha = torch.clamp(h_alpha, 0, 1)

    adaround_weights = torch.floor(inputs * scale).to(device) + h_alpha

    outputs = torch.clamp(adaround_weights, min_bound, max_bound)

    if min_amax <= epsilon:
        scale[zero_amax_mask] = 1.  # Return 1 makes more sense for values quantized to 0 with amax=0

    if input_dtype == torch.half:
        outputs = outputs.half()

    outputs = outputs / scale.to(inputs.dtype)

    return outputs

def compute_loss(adaround_output, output, alpha, opt_params, iteration):
    recon_loss = AdaroundLoss.compute_recon_loss(adaround_output, output)
    round_loss = AdaroundLoss.compute_round_loss(alpha, opt_params, iteration)
    return recon_loss + round_loss



In [39]:
import pickle
from torch.utils.data import Dataset, DataLoader

class LayerInputDataset(Dataset):
    def __init__(self, layer_activations):
        """
        Args:
            layer_activations (Tensor): Tensor containing activations for a specific layer.
        """
        self.layer_activations = layer_activations

    def __len__(self):
        return len(self.layer_activations)

    def __getitem__(self, idx):
        return self.layer_activations[idx]



def load_activations(directory):
    activations = {}
    for filename in os.listdir(directory):
        if filename.endswith('.pkl'):
            layer_name = filename.replace('.pkl', '')
            with open(os.path.join(directory, filename), 'rb') as file:
                activations[layer_name] = pickle.load(file)
    return activations

In [40]:
names = ['conv_0']
layers = [(model_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].conv,
          model_full_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].layer_norm,
          model_full_quantized.mods.encoder.wav2vec2.model.feature_extractor.conv_layers[0].activation
          )]
directory = './'
activations = load_activations(directory)
layer_datasets = {}
for layer_name, layer_activation in activations.items():
    layer_datasets[layer_name] = LayerInputDataset(layer_activation)

In [41]:
import numpy as np
from tqdm import tqdm
alphas = {}

for name, layers in zip(names,layers):
    alphas[name] = optimize_adaround(layer = layers[0], norm = layers[1], act = layers[2],
                                     dataset = layer_datasets[name],
                                     device='cuda'
                                    )
with open('alphas.pkl', 'wb') as f:
    pickle.dump(alphas, f)

Adaround Optimization:   0%|                                                            | 1/4000 [00:01<1:40:29,  1.51s/it]

Iteration 1/4000, Average Loss: 4.956180447691482e-08


Adaround Optimization:   0%|                                                            | 2/4000 [00:02<1:16:23,  1.15s/it]

Iteration 2/4000, Average Loss: 8.21019574917016e-12


Adaround Optimization:   0%|                                                            | 3/4000 [00:03<1:08:12,  1.02s/it]

...

Iteration 3998/4000, Average Loss: 0.001840039060334675


Adaround Optimization: 100%|████████████████████████████████████████████████████████▉| 3999/4000 [1:05:37<00:00,  1.05it/s]

Iteration 3999/4000, Average Loss: 0.001840039060334675


Adaround Optimization: 100%|█████████████████████████████████████████████████████████| 4000/4000 [1:05:38<00:00,  1.02it/s]

Iteration 4000/4000, Average Loss: 0.001840039060334675
